In [89]:
from pymongo import MongoClient
import pandas as pd
import plotly.express as plty

In [90]:
def connect_database(database_name : str) :
    """Create connexion to Mongo Database

    Args:
        database_name (str): Name of the database

    Returns:
        Client connexion
    """
    client = MongoClient()
    return client[database_name]

def get_collection(connection, collection_name : str) :
    """Connect to a Mongo collection

    Args:
        connection : Mongo client connection
        collection_name (str): Name of the collection

    Returns:
        _type_: The collection
    """
    return connection[collection_name]

In [91]:
def read_and_create_dataframe(collection) -> pd.DataFrame :
    """Read a collection with League of Legends JSON data and create the associated dataframe
    Each row correspond to (match,participant) key.

    Args:
        collection (_type_): Mongo collection

    Returns:
        pd.DataFrame: DataFrame of the JSON data
    """
    df = pd.DataFrame()
    for game in collection.find() :
        df = pd.concat([df,pd.json_normalize(game)])
    df = df.explode('participants').reset_index(drop=True)
    df_participants = pd.json_normalize(df['participants'])
    df = pd.concat([df.drop(columns='participants'),df_participants],axis = 1)
    df['VISION_WARDS_BOUGHT_IN_GAME'] = df['VISION_WARDS_BOUGHT_IN_GAME'].astype('int')
    return df

In [92]:
connect = connect_database('lol_match_database')
scrim_matches = get_collection(connect,"scrim_matches")
data_scrim_matches = read_and_create_dataframe(scrim_matches)
data_scrim_matches

,_id,matchId,gameDuration,gameVersion,ALL_IN_PINGS,ASSIST_ME_PINGS,ASSISTS,BAIT_PINGS,BARON_KILLS,BARRACKS_KILLED,...,VISION_WARDS_BOUGHT_IN_GAME,WARD_KILLED,WARD_PLACED,WARD_PLACED_DETECTOR,WAS_AFK,WAS_AFK_AFTER_FAILED_SURRENDER,WAS_EARLY_SURRENDER_ACCOMPLICE,WAS_LEAVER,WAS_SURRENDER_DUE_TO_AFK,WIN
0,676045f051f627b84cbbdedd,6893264796,1960237,14.7.571.9528,0,0,4,None,0,0,...,4,0,15,4,0,0,0,0,0,Fail
1,676045f051f627b84cbbdedd,6893264796,1960237,14.7.571.9528,3,6,10,None,0,0,...,10,10,24,8,0,0,0,0,0,Fail
2,676045f051f627b84cbbdedd,6893264796,1960237,14.7.571.9528,0,0,13,None,0,0,...,3,3,7,3,0,0,0,0,0,Fail
3,676045f051f627b84cbbdedd,6893264796,1960237,14.7.571.9528,0,0,5,None,0,0,...,2,5,8,2,0,0,0,0,0,Fail
4,676045f051f627b84cbbdedd,6893264796,1960237,14.7.571.9528,0,0,10,None,0,0,...,5,10,52,5,0,0,0,0,0,Fail
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
85,6760461051f627b84cbbdee5,Unknown,1346472,14.21.629.0951,0,0,1,None,0,0,...,0,2,7,0,0,0,0,0,0,Fail
86,6760461051f627b84cbbdee5,Unknown,1346472,14.21.629.0951,0,0,2,None,0,0,...,2,2,4,2,0,0,0,0,0,Fail
87,6760461051f627b84cbbdee5,Unknown,1346472,14.21.629.0951,0,0,0,None,0,0,...,2,2,8,2,0,0,0,0,0,Fail
88,6760461051f627b84cbbdee5,Unknown,1346472,14.21.629.0951,0,2,2,None,0,0,...,0,1,5,0,0,0,0,0,0,Fail


### Team dico

In [93]:
team_scald_dico = {
    "TOP" : ["62baa689-290d-5daa-be1a-a9da42fca824",],
    "JUNGLE" : ["90cb66b0-18fd-5689-bf52-c5a80c88cdc5"],
    "MIDDLE" : ["52334604-2f25-5556-8284-90370c43f6eb"],
    "BOTTOM": ["9af56d81-a4c0-5447-b465-dd203dd80c6f"],
    "UTILITY" : ["950ff368-e5a1-572e-84c1-824a6364dd3b","384dbdbf-bc03-5475-9002-865c1b4c2ae1"]
}

### 

In [94]:
def filter_data_on_team(data : pd.DataFrame,team_dict : dict) -> pd.DataFrame :
    """Filter data based on a team PUUID dictionnary

    Args:
        data (pd.DataFrame): Input data
        team_dict (dict): Dictionnary containing PUUID. Example = {"TOP" : ["9df5d86"], "JUNGLE" : ...}

    Returns:
        pd.DataFrame: The filtered DataFrame
    """
    return data.loc[data['PUUID'].apply(lambda puuid: any(puuid in sublist for sublist in team_dict.values()))]

In [95]:
scl = filter_data_on_team(data_scrim_matches,team_scald_dico)

In [96]:
def get_winrate_by_side(data : pd.DataFrame, chart = False) :
    """Compute winrate in blue and red side for a dataFrame (usage with team filter)

    Args:
        data (pd.DataFrame): Filtered team data
        chart : Show the figure plot

    Returns:
        _type_: Return dict of both winrate side
    """
    winrate_blue = data.loc[(data['WIN']=='Win') & (data['TEAM']=='100'),'WIN'].count() / len(data.loc[data['TEAM']=='100']) * 100
    winrate_red = data.loc[(data['WIN']=='Win') & (data['TEAM']=='200'),'WIN'].count() / len(data.loc[data['TEAM']=='200']) * 100

    if chart : 
        fig = plty.bar(x=['Blue','Red'], y=[winrate_blue,winrate_red], labels={"x" : "Side", "y" : "Winrate (%)"})
        fig.show()
    return {"blue" : float(winrate_blue) , "red" : float(winrate_red)}

In [97]:
get_winrate_by_side(scl, chart=True)

C:\Users\benoi\AppData\Local\Temp\ipykernel_18632\2759414159.py:11: RuntimeWarning:

invalid value encountered in longlong_scalars

C:\Users\benoi\AppData\Local\Temp\ipykernel_18632\2759414159.py:12: RuntimeWarning:

invalid value encountered in longlong_scalars



{'blue': nan, 'red': nan}

In [98]:
def table_winrate_champs(data : pd.DataFrame) :
    """Retrieve and groupby champion from the dataFrame and get number of game and number of win. Use this with filtered data.

    Args:
        data (pd.DataFrame): The filtered DataFrame

    Returns:
        list : List from TOP to SUPPORT champions game and winrate
    """
    positions = ["TOP","JUNGLE","MIDDLE","BOTTOM","UTILITY"]
    top_to_bot_champs = []
    for position in positions :

        all = data.loc[data['TEAM_POSITION'] == position].groupby("SKIN")['WIN'].count()
        win = data.loc[(data['TEAM_POSITION'] == position) & (data['WIN'] == 'Win')].groupby("SKIN")['WIN'].count()

        df_player = pd.DataFrame(data= {'Count' : all,'Win' : win}).fillna(0)
        df_player = df_player.astype({'Win' : int})


        top_to_bot_champs.append(df_player)    
    return top_to_bot_champs

In [99]:
table_winrate_champs(scl)

[Empty DataFrame
 Columns: [Count, Win]
 Index: [],
 Empty DataFrame
 Columns: [Count, Win]
 Index: [],
 Empty DataFrame
 Columns: [Count, Win]
 Index: [],
 Empty DataFrame
 Columns: [Count, Win]
 Index: [],
 Empty DataFrame
 Columns: [Count, Win]
 Index: []]

In [100]:
def get_nb_pink_bought(data : pd.DataFrame, chart=False) -> list : 
    """Function to get the median number of pink bought in all games role by role.

    Args:
        data (pd.DataFrame): The filtered DataFrame
        chart (boolean) : Display the chart

    Returns:
        list: List role by role (top - supp) of the median of pink
    """
    
    positions = ["TOP","JUNGLE","MIDDLE","BOTTOM","UTILITY"]
    top_to_bot_pink_median = []

    for position in positions :
        top_to_bot_pink_median.append(data.loc[data['TEAM_POSITION'] == position, 'VISION_WARDS_BOUGHT_IN_GAME'].median())
    
    if chart :
        fig = plty.bar(x=positions, y=top_to_bot_pink_median, labels={"x" : "Positions", "y" : "Median nb of pink"})
        fig.show()
    return top_to_bot_pink_median

In [101]:
get_nb_pink_bought(scl, chart=True)

[nan, nan, nan, nan, nan]

In [102]:
def calculate_matchup_winrate(data: pd.DataFrame, team_dict: dict, role: str, enemy_dict: str = None, position_filter: str = None) -> pd.DataFrame:
    """
    Calculate the winrate of a specific role in matchups.

    Args:
        data (pd.DataFrame): The DataFrame containing the match data.
        team_dict (dict): Dictionary mapping roles to player PUUIDs.
        role (str): Role to analyze (e.g., "TOP", "JUNGLE").
        chart (bool): Whether to display the result as a chart (currently not implemented).
        enemy_dict (str): Optionally specify an enemy champion to filter matchups. If None, all enemies are included.
        position_filter (str): Optionally specify a position to filter the allies by (e.g., "TOP"). If None, no position filter is applied.

    Returns:
        pd.DataFrame: A DataFrame containing winrates for each matchup.
    """
    # Step 1: Retrieve allied data for the specified position and optionally filter by position
    if position_filter:
        role_data = data[(data['PUUID'].isin(team_dict[role])) & (data['TEAM_POSITION'] == position_filter)][['_id', 'SKIN', 'WIN']].copy()
    else:
        role_data = data[data['PUUID'].isin(team_dict[role])][['_id', 'SKIN', 'WIN']].copy()

    # Step 2: Retrieve the direct opponents in the specified position
    opponent_data = data[
        (data['TEAM_POSITION'] == role) & 
        (~data['PUUID'].isin(team_dict[role]))  # Exclude allies
    ][['_id', 'SKIN']].rename(columns={'SKIN': 'ENEMY_CHAMPION'}).copy()

    # If a specific enemy champion is provided, filter the opponents
    if enemy_dict:
        opponent_data = opponent_data[opponent_data['ENEMY_CHAMPION'] == enemy_dict]

    # Step 3: Merge the allied data with the opponent data based on "_id"
    merged_data = pd.merge(role_data, opponent_data, on='_id')

    # Step 4: Group by to calculate the number of games played and the wins
    matchup_stats = merged_data.groupby(['SKIN', 'ENEMY_CHAMPION']).agg(
        GAMES=('WIN', 'count'),
        WINS=('WIN', lambda x: (x == 'Win').sum())  # Count the wins
    ).reset_index()

    # Step 5: Calculate the winrate
    matchup_stats['WINRATE'] = (matchup_stats['WINS'] / matchup_stats['GAMES']) * 100

    # Step 6: Sort by the number of games played (GAMES)
    matchup_stats = matchup_stats.sort_values(by='GAMES', ascending=False)

    # Display the result as a table
    display(matchup_stats) 
   

    # Return the matchup statistics
    return matchup_stats




In [103]:
matchup_stats = calculate_matchup_winrate(data_scrim_matches, team_scald_dico, role="JUNGLE", position_filter="JUNGLE", enemy_dict=None)

,SKIN,ENEMY_CHAMPION,GAMES,WINS,WINRATE


In [104]:
def calculate_duo_winrate(filtered_data: pd.DataFrame, roles: tuple = ("MIDDLE", "JUNGLE")) -> pd.DataFrame:
    """
    Calculate winrate for champion duos in specified roles using pre-filtered data.

    Args:
        filtered_data (pd.DataFrame): The DataFrame already filtered for the team's matches.
        roles (tuple): Roles to analyze (e.g., ("MIDDLE", "JUNGLE")).

    Returns:
        pd.DataFrame: A DataFrame containing winrates for each duo.
    """
    # Extract the data for each role directly
    role1_data = filtered_data[filtered_data['TEAM_POSITION'] == roles[0]][['_id', 'SKIN', 'WIN']].rename(
        columns={'SKIN': f"{roles[0]}_CHAMPION"}
    )
    role2_data = filtered_data[filtered_data['TEAM_POSITION'] == roles[1]][['_id', 'SKIN']].rename(
        columns={'SKIN': f"{roles[1]}_CHAMPION"}
    )

    # Merge the two roles' data on '_id'
    duo_data = pd.merge(role1_data, role2_data, on="_id")

    # Group by the duo of champions and calculate stats
    duo_stats = duo_data.groupby([f"{roles[0]}_CHAMPION", f"{roles[1]}_CHAMPION"]).agg(
        GAMES=('WIN', 'count'),
        WINS=('WIN', lambda x: (x == 'Win').sum())  # Count the wins
    ).reset_index()

    # Calculate the winrate
    duo_stats['WINRATE'] = (duo_stats['WINS'] / duo_stats['GAMES']) * 100

    # Sort by the number of games played
    duo_stats = duo_stats.sort_values(by='GAMES', ascending=False)

    # Display the result
    display(duo_stats)

    # Return the duo statistics
    return duo_stats



In [105]:
duo_stats = calculate_duo_winrate(filter_data_on_team(data_scrim_matches,team_scald_dico), roles=("JUNGLE", "MIDDLE"))

,JUNGLE_CHAMPION,MIDDLE_CHAMPION,GAMES,WINS,WINRATE
